In [63]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from utils.Dataset_modification import *
from utils.compute_model_performance import *
import lightgbm as xgb

X_train = pd.read_csv(datasets+'X_train.csv') #datasets is defined in utils
Y_train=pd.read_csv(datasets+'Y_train.csv')

X_train=X_train.drop(["DE_NET_IMPORT","FR_NET_IMPORT","DE_FR_EXCHANGE",'DAY_ID'],axis=1)

country_mapping = {'FR': 0, 'DE': 1}
X_train['COUNTRY'] = X_train['COUNTRY'].map(country_mapping)
Y_train['Rank']= Y_train['TARGET'].rank().astype(np.int16)
Y_train['Rank'] = pd.qcut(Y_train.Rank, 30).cat.codes
X_train.fillna(X_train.mean(),inplace=True)
features_columns=X_train.drop('ID',axis=1).columns
Y_train=Y_train.drop(['ID','TARGET'], axis=1)

In [64]:
X_feat=X_train[features_columns]
X_feat

,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_GAS,FR_GAS,DE_COAL,FR_COAL,...,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET
0,0,0.210099,-0.427458,0.606523,-0.256332,0.692860,0.441238,-0.213766,0.740627,0.288782,...,-0.444661,-0.172680,-0.556356,-0.790823,-0.283160,-1.069070,-0.063404,0.339041,0.124552,-0.002445
1,0,-0.022399,-1.003452,0.022063,-0.573520,-1.130838,0.174773,0.426940,-0.170392,-0.762153,...,-1.183194,-1.240300,-0.770457,1.522331,0.828412,0.437419,1.831241,-0.659091,0.047114,-0.490365
2,0,1.395035,1.978665,-1.021305,-0.622021,-1.682587,2.351913,2.122241,1.572267,0.777053,...,1.947273,-0.480700,-0.313338,0.431134,0.487608,0.684884,0.114836,0.535974,0.743338,0.204952
3,1,-0.983324,-0.849198,0.839586,-0.270870,0.563230,0.487818,0.194659,-1.473817,-0.786025,...,-0.976974,-1.114838,-0.507570,-0.499409,-0.236249,0.350938,-0.417514,0.911652,-0.296168,1.073948
4,0,0.143807,-0.617038,0.924990,-0.256332,0.990324,0.238693,-0.240862,1.003734,-0.274975,...,-0.526267,-0.541465,-0.424550,-1.088158,-1.011560,0.614338,0.729495,0.245109,1.526606,2.614378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489,1,1.529204,1.106682,1.855327,-0.218658,1.450426,1.810665,1.388269,0.359723,-0.294001,...,0.509514,-0.037831,0.019357,0.109480,0.123099,0.009451,0.008404,0.876984,0.819520,1.320373
1490,0,1.618582,1.752840,-0.611392,0.449153,-0.152146,1.972779,1.558300,0.561356,0.230746,...,1.666252,-0.037831,0.019357,0.109480,0.123099,0.009451,0.008404,0.932633,-0.085690,0.356356
1491,1,0.856399,0.489199,0.255778,-1.531544,-0.829568,2.108764,1.866399,1.072553,-0.180117,...,0.358120,0.207905,0.404763,-0.594595,0.894011,0.256338,0.402316,-1.112899,-0.237835,0.067152
1492,0,0.560689,-0.343777,0.830239,-0.304856,1.210230,-0.003973,0.869742,-0.436935,-0.772801,...,-0.184862,-0.682815,-0.390304,-0.972088,-1.501930,1.215528,1.338708,0.962812,-5.392852,-0.843812


In [69]:
Y_train['Rank']


0       16
1       10
2        8
3        6
4       11
        ..
1489     8
1490    12
1491    20
1492     3
1493     6
Name: Rank, Length: 1494, dtype: int8

In [66]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_feat, Y_train, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [49]:
X_train

,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_GAS,FR_GAS,DE_COAL,FR_COAL,...,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET
1193,0,0.406000,-0.634804,1.513080,-1.173137,1.316583,0.426373,-0.057921,-0.454895,-0.775492,...,-0.751873,0.392121,-1.648699,-0.653331,-0.432017,1.208224,0.776142,0.807354,1.170828,2.566665
36,0,0.031579,-0.693302,0.577114,-0.467340,1.175802,0.532865,0.201683,1.110380,0.156575,...,-0.619215,2.383630,0.798208,0.071497,0.693684,1.100208,0.713570,0.338003,-1.207689,0.644785
1167,0,0.212648,-0.107135,0.646519,-0.408507,0.520912,0.624250,-0.302384,-1.274925,-0.773642,...,-0.177433,-0.450105,-0.192482,-0.835374,-0.663243,-1.318402,-0.892269,-1.477827,-0.996594,-0.383438
62,0,-0.163169,-1.000510,0.844216,-1.248088,-0.483415,1.231830,0.149893,-0.551608,-0.784101,...,-0.916060,0.028255,-0.704780,-0.715431,-0.720297,0.366905,0.281706,0.107630,0.326670,0.726900
1347,0,0.678396,-0.188685,-0.046277,0.073954,0.015142,0.654267,0.023003,-0.788914,-0.794729,...,-0.506559,0.457645,0.054701,-0.377646,-0.284845,0.124144,-0.128992,0.053749,0.129690,0.579926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,0,0.166886,-0.574960,0.216434,-0.256332,-0.072643,-0.416947,-1.220452,0.269065,-0.794950,...,-0.433362,-0.079326,-0.135037,-0.880549,-1.242912,2.287095,1.978075,0.228896,-1.191729,0.021316
43,0,-0.328286,-1.062255,1.380464,-1.046122,1.002243,0.544008,-0.730992,-1.183566,-0.455367,...,-1.130258,0.974454,-0.069416,-0.138918,-0.695013,-0.634046,-0.168491,0.122818,0.220077,5.453331
77,1,0.463015,-0.318192,-0.105749,1.144960,0.100153,1.067490,0.235612,-0.889685,-0.781127,...,-0.675188,0.989594,0.183246,-0.010053,-0.150271,-0.061771,-0.643484,-0.288695,0.236041,-0.071109
380,0,-0.504973,-0.911500,0.572897,-1.134687,0.269540,1.535793,0.215930,-1.172011,-0.778530,...,-0.921322,1.140074,-0.230571,-0.506865,-0.621892,0.073140,-0.447733,0.422695,0.177636,0.861586


In [1]:
import numpy as np
import pandas as pd
import lightgbm



In [67]:
model = lightgbm.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    force_row_wise=True,
    min_data_in_leaf=10,
    max_depth=-1,
    min_sum_hessian_in_leaf=2,
    boosting_type="dart",
    n_estimators=100,
    importance_type='gain'
)

In [71]:
model.fit(
    X=X_train,
    y=list(y_train['Rank']),
    group=[len(X_train)],
    eval_set=[(X_val, y_val['Rank'])],
    eval_group=[len(X_val)],
    eval_at=10)

[LightGBM] [Warning] min_sum_hessian_in_leaf is set=2, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=2
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=2, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=2
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Info] Total Bins 7384
[LightGBM] [Info] Number of data points in the train set: 956, number of used features: 30
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=2, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=2
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10


TypeError: Wrong type(int) for group.
It should be list, numpy 1-D array or pandas Series